In [1]:
import scipy.io
import numpy as np
import networkx as nx

# 读取.mat文件
mat = scipy.io.loadmat('/Users/shenxiaoyu/Desktop/Brain data/shamfc_s1.mat')

# 提取邻接矩阵数据
shamfc_s1 = mat['shamfc_s1']

# 检查数据形状
print("Data shape:", shamfc_s1.shape)  # 应该输出 (20, 360, 360)

# 创建一个空的列表来存储每个被试的图结构
graphs = []

# 遍历每个被试，转换邻接矩阵为networkx图结构
for i in range(shamfc_s1.shape[0]):
    adj_matrix = shamfc_s1[i]
    G = nx.from_numpy_matrix(adj_matrix)
    graphs.append(G)

# 现在，graphs列表包含了每个被试的图结构
# 你可以对这些图进行进一步的分析

#检查图的信息
print(nx.info(graphs[0]))
is_directed = nx.is_directed(G)
print(f"The graph is {'directed' if is_directed else 'undirected'}.")
if is_directed:
    is_strongly_connected = nx.is_strongly_connected(G)
    print(f"The graph is {'strongly connected' if is_strongly_connected else 'not strongly connected'}.")
    
    is_weakly_connected = nx.is_weakly_connected(G)
    print(f"The graph is {'weakly connected' if is_weakly_connected else 'not weakly connected'}.")
else:
    is_connected = nx.is_connected(G)
    print(f"The graph is {'connected' if is_connected else 'not connected'}.")


Data shape: (20, 360, 360)
Graph with 360 nodes and 64980 edges
The graph is undirected.
The graph is connected.


In [2]:

import scipy.io

# 读取.mat文件
file_path = '/Users/shenxiaoyu/Desktop/Brain data/shamfc_s1.mat'
mat = scipy.io.loadmat(file_path)

# 打印文件中的所有变量
print(mat.keys())

# 提取矩阵
key = [k for k in mat.keys() if not k.startswith('__')][0]  # 提取主要数据的key
matrix = mat[key]
print(f"Shape of the matrix: {matrix.shape}")

dict_keys(['__header__', '__version__', '__globals__', 'shamfc_s1'])
Shape of the matrix: (20, 360, 360)


In [ ]:
import scipy.io
import numpy as np
import networkx as nx
import os
from scipy.stats import ttest_ind

# 定义文件路径列表
folder_path = '/Users/shenxiaoyu/Desktop/Brain data'
file_paths = [
    'shamfc_s1.mat', 'shamfc_s2.mat', 'shamfc_s3.mat',
    'shamzfc_s1.mat', 'shamzfc_s2.mat', 'shamzfc_s3.mat',
    'tacs6fc_s1.mat', 'tacs6fc_s2.mat', 'tacs6fc_s3.mat',
    'tacs6zfc_s1.mat', 'tacs6zfc_s2.mat', 'tacs6zfc_s3.mat',
    'tacs10fc_s1.mat', 'tacs10fc_s2.mat', 'tacs10fc_s3.mat',
    'tacs10zfc_s1.mat', 'tacs10zfc_s2.mat', 'tacs10zfc_s3.mat',
    'tdcsfc_s1.mat', 'tdcsfc_s2.mat', 'tdcsfc_s3.mat',
    'tdcszfc_s1.mat', 'tdcszfc_s2.mat', 'tdcszfc_s3.mat'
]

# 创建一个字典来存储每种条件下的图
graphs = {
    'sham': {'s1': [], 's2': [], 's3': []},
    'tacs6': {'s1': [], 's2': [], 's3': []},
    'tacs10': {'s1': [], 's2': [], 's3': []},
    'tdcs': {'s1': [], 's2': [], 's3': []}
}

# 数据预处理：读取文件并提取 zfc 矩阵
for file_name in file_paths:
    file_path = os.path.join(folder_path, file_name)
    mat = scipy.io.loadmat(file_path)
    key = [k for k in mat.keys() if not k.startswith('__')][0]  # 提取主要数据的key
    matrix = mat[key]
    
    # 获取条件和时间点
    base_name = file_name.replace('.mat', '')
    condition_time, matrix_type = base_name.split('_')
    condition, time_point = condition_time[:-2], condition_time[-2:]
    
    if matrix_type == 'zfc':
        # 转换为图结构并存储
        for i in range(matrix.shape[0]):
            adj_matrix = matrix[i]
            G = nx.from_numpy_matrix(adj_matrix)
            graphs[condition][time_point].append(G)

# 定义函数计算图论指标
def compute_graph_metrics(G):
    return {
        'degree_centrality': np.mean(list(nx.degree_centrality(G).values())),
        'betweenness_centrality': np.mean(list(nx.betweenness_centrality(G).values())),
        'eigenvector_centrality': np.mean(list(nx.eigenvector_centrality(G).values())),
        'clustering_coefficient': nx.average_clustering(G),
        'modularity': nx.algorithms.community.quality.modularity(G, nx.community.greedy_modularity_communities(G))
    }

# 计算每个图的指标
metrics = {
    'sham': {'s1': [], 's2': [], 's3': []},
    'tacs6': {'s1': [], 's2': [], 's3': []},
    'tacs10': {'s1': [], 's2': [], 's3': []},
    'tdcs': {'s1': [], 's2': [], 's3': []}
}

for condition in graphs:
    for time_point in graphs[condition]:
        for G in graphs[condition][time_point]:
            metrics[condition][time_point].append(compute_graph_metrics(G))

# 比较分析
def compare_conditions(metrics, condition1, condition2, time_point, metric_name):
    data1 = [m[metric_name] for m in metrics[condition1][time_point]]
    data2 = [m[metric_name] for m in metrics[condition2][time_point]]
    t_stat, p_value = ttest_ind(data1, data2)
    return t_stat, p_value

# 示例：比较 sham 和 tacs6 在 s1 时的度中心性
t_stat, p_value = compare_conditions(metrics, 'sham', 'tacs6', 's1', 'degree_centrality')
print(f"T-statistic: {t_stat}, P-value: {p_value}")

# 根据需要对所有条件和时间点进行比较


In [ ]:
import scipy.io
import numpy as np
import networkx as nx
import os

# 定义文件路径列表
folder_path = '/Users/shenxiaoyu/Desktop/Brain data'
file_paths = [
    'shamfc_s1.mat', 'shamfc_s2.mat', 'shamfc_s3.mat',
    'shamzfc_s1.mat', 'shamzfc_s2.mat', 'shamzfc_s3.mat',
    'tacs6fc_s1.mat', 'tacs6fc_s2.mat', 'tacs6fc_s3.mat',
    'tacs6zfc_s1.mat', 'tacs6zfc_s2.mat', 'tacs6zfc_s3.mat',
    'tacs10fc_s1.mat', 'tacs10fc_s2.mat', 'tacs10fc_s3.mat',
    'tacs10zfc_s1.mat', 'tacs10zfc_s2.mat', 'tacs10zfc_s3.mat',
    'tdcsfc_s1.mat', 'tdcsfc_s2.mat', 'tdcsfc_s3.mat',
    'tdcszfc_s1.mat', 'tdcszfc_s2.mat', 'tdcszfc_s3.mat'
]

# 创建一个字典来存储每种条件下的图
graphs = {
    'sham': {'s1': [], 's2': [], 's3': []},
    'tacs6': {'s1': [], 's2': [], 's3': []},
    'tacs10': {'s1': [], 's2': [], 's3': []},
    'tdcs': {'s1': [], 's2': [], 's3': []}
}

# 数据预处理：读取文件并提取 zfc 矩阵
for file_name in file_paths:
    file_path = os.path.join(folder_path, file_name)
    mat = scipy.io.loadmat(file_path)
    key = [k for k in mat.keys() if not k.startswith('__')][0]  # 提取主要数据的key
    matrix = mat[key]
    
    # 获取条件和时间点信息
    base_name = file_name.replace('.mat', '')
    condition_time, matrix_type = base_name.split('_')
    condition, time_point = condition_time[:-2], condition_time[-2:]
    
    if matrix_type == 'zfc':
        # 转换为图结构并存储
        for i in range(matrix.shape[0]):
            adj_matrix = matrix[i]
            G = nx.from_numpy_matrix(adj_matrix)
            graphs[condition][time_point].append(G)

# 定义函数计算图论指标
def compute_graph_metrics(G):
    return {
        'degree_centrality': np.mean(list(nx.degree_centrality(G).values())),
        'betweenness_centrality': np.mean(list(nx.betweenness_centrality(G).values())),
        'eigenvector_centrality': np.mean(list(nx.eigenvector_centrality(G).values())),
        'clustering_coefficient': nx.average_clustering(G),
        'modularity': nx.algorithms.community.quality.modularity(G, nx.community.greedy_modularity_communities(G))
    }

# 计算每个图的指标
metrics = {
    'sham': {'s1': [], 's2': [], 's3': []},
    'tacs6': {'s1': [], 's2': [], 's3': []},
    'tacs10': {'s1': [], 's2': [], 's3': []},
    'tdcs': {'s1': [], 's2': [], 's3': []}
}

for condition in graphs:
    for time_point in graphs[condition]:
        for G in graphs[condition][time_point]:
            metrics[condition][time_point].append(compute_graph_metrics(G))

# 比较分析
def compare_conditions(metrics, condition1, condition2, time_point, metric_name):
    data1 = [m[metric_name] for m in metrics[condition1][time_point]]
    data2 = [m[metric_name] for m in metrics[condition2][time_point]]
    t_stat, p_value = ttest_ind(data1, data2)
    return t_stat, p_value

# 示例：比较 sham 和 tacs6 在 s1 时的度中心性
t_stat, p_value = compare_conditions(metrics, 'sham', 'tacs6', 's1', 'degree_centrality')
print(f"T-statistic: {t_stat}, P-value: {p_value}")

# 根据需要对所有条件和时间点进行比较
